In [91]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [92]:
# Create the optimization model
model = gb.Model("Question 2: Puff")

In [93]:
max_demand_regular = [72, 70, 123, 93, 51, 41]
slope_regular = [15, 12, 13, 8, 20, 42]
max_demand_jelly = [66, 76, 116, 86, 56, 46]
slope_jelly = [14, 13, 11, 8, 24, 28]

In [94]:
p_regular = model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Regular Price")
p_jelly = model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Jelly Price")
model.setObjective(gb.quicksum(((p_regular[i] - (2*0.33))*(max_demand_regular[i] - slope_regular[i]*p_regular[i])) + ((p_jelly[i] - (2*0.46))*(max_demand_jelly[i] - slope_jelly[i]*p_jelly[i])) for i in range(6)), GRB.MAXIMIZE)

In [95]:
# Price Constraint
for n in range(2):  
    model.addConstr(p_regular[n] <= p_regular[n + 1], f"Regular Price Constraint {n}")

for n in range(2):  
    model.addConstr(p_jelly[n] <= p_jelly[n + 1], f"Jelly Price Constraint {n}")

In [96]:
# Price Constraint
for n in range(2,5):  
    model.addConstr(p_regular[n] >= p_regular[n + 1], f"Regular Price Constraint {n}")

for n in range(2,5):  
    model.addConstr(p_jelly[n] >= p_jelly[n + 1], f"Jelly Price Constraint {n}")

In [97]:
model.addConstr(gb.quicksum(max_demand_regular[i] - (slope_regular[i]*p_regular[i]) for i in range(6)) <= (222/2), "Regular Supply Constraint")
model.addConstr(gb.quicksum(max_demand_regular[i] - (slope_regular[i]*p_regular[i]) for i in range(6)) >= (0), "Regular Supply Constraint")

model.addConstr(gb.quicksum(max_demand_jelly[i] - slope_jelly[i]*p_jelly[i] for i in range(6)) <= (274/2), "Jelly Supply Constraint")
model.addConstr(gb.quicksum(max_demand_jelly[i] - slope_jelly[i]*p_jelly[i] for i in range(6)) >= (0), "Jelly Supply Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [98]:
#for n in range(6):  
    #model.addConstr(1.05*p_regular[n] <= p_jelly[n], "5% Constraint")

In [99]:
# Solve our model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 14 rows, 12 columns and 44 nonzeros
Model fingerprint: 0x69a2f00d
Model has 12 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [6e+01, 1e+02]
  QObjective range [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 5e+02]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 12 rows, 14 columns, 34 nonzeros
Presolved model has 12 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.800e+01
 Factor NZ  : 4.200e+01
 Factor Ops : 1.820e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Co

In [100]:
# Value of the objective function
print("Profit: ", round(model.objVal,2))

Profit:  1170.83


In [101]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Regular Price[0]      3.43636 
Regular Price[1]      3.95303 
Regular Price[2]      6.17922 
Regular Price[3]      6.17922 
Regular Price[4]      2.31136 
Regular Price[5]      1.52446 
Jelly Price[0]      3.23469 
Jelly Price[1]      3.80063 
Jelly Price[2]      6.19334 
Jelly Price[3]      6.19334 
Jelly Price[4]      2.04422 
Jelly Price[5]      1.69898 
None
